In [1]:
import numpy as np
import csv
import json
import os 
import datetime
import matplotlib.pyplot as plt
import seaborn as sb

from IPython.display import Image, display, HTML

import pprint 
pp = pprint.PrettyPrinter(indent=4)


In [2]:
import config
import utils


In [3]:
#
# Summary log type
#
class SummaryLog:
    """
    Represents one summary log file.
    """
    def __init__(self):
        self.data = []
                
    @staticmethod
    def parse_file(full_path, filename):

        log = SummaryLog()
        fpth = os.path.join(full_path, filename)

        with open(fpth, 'r') as ifs:    
            line = ifs.readline()
            while (line):
                log.parse_line(line)
                line = ifs.readline()

        return log
    
    def parse_line(self, line):
        self.data.append(line)

In [4]:
#
# JSON log type
#
class JsonLog:
    """
    Represents one JSON log file.
    """
    def __init__(self):
        pass
        
    @staticmethod
    def fix_JSON(JSON_text, filename):

        if (len(JSON_text) == 0):
            raise Exception("File {} is empty!".format(filename))

        #
        # Remove an extra comma at the end of the file
        #
        comma_idx = JSON_text.rfind(",")
        text_len = len(JSON_text)
        if (comma_idx >= text_len - 3):
            JSON_text = JSON_text[0:comma_idx] + "\n"


        #
        # Wrap submit logs with an array, cause there may be multiple submits at the same millisecond
        #
        if (filename.endswith("submit.json")):
            JSON_text = "[{}]".format(JSON_text)


        fixed = JSON_text

        try:
            parsed = json.loads(fixed)
            return fixed
        except:
            print(fixed)
            raise Exception("nende")

    @staticmethod
    def parse_file(full_path, filename):
        fpth = os.path.join(full_path, filename)
        parsed = None
        with open(fpth, 'r') as ifs:    
            try:

                parsed = json.load(ifs)
                if (filename.endswith("submit.json") and type(parsed) is dict):
                    raise Exception("!!")
            except:
                ifs.seek(0)
                text = ifs.read()

                parsed = JsonLog.fix_JSON(text, filename)
                ifs.close()

                stinfo = os.stat(fpth)                
                with open(fpth, 'w') as ofs:
                    ofs.write(parsed)
                    print("W: Rewritten the file {}!".format(filename))
                os.utime(fpth ,(stinfo.st_atime, stinfo.st_mtime))
        return parsed      

In [13]:
class Data:
    def __init__(self):
        self.verbose = False
        
        self.tasks = TaskDefs.parse_tasks(config.TASKS_JSON, config.TASKS_STARTS, config.TASK_MAPPING, config.thumbs_list_filepath(), verbose=self.verbose)
        
    def parse_team(self, team_name: str, team_names: list, verbose = False, validate_fix=False, generate_DRES=False):
        self.verbose = verbose
        
        print("==============\nTEAM: {}\n".format(team_name))
        
        ### main()
        if (validate_fix):
            self.validate_and_fix_input_data(team_name, team_names)
        
        self.calculate_server_ts_diff(team_name, team_names)
        
        if (generate_DRES):
            self.generate_DRES_logs(team_name, team_names)
        
        
        ###
        
        self.verbose = False
     
    
    def validate_and_fix_input_data(self, team_name: str, team_names: list):
        print("%%% VALIDATING & FIXING %%%")
            
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            self.parsed[team_name] = self.validate_user(user_name, path)
            
        print("%%% DONE! %%%")
    
    def calculate_server_ts_diff(self, team_name: str, team_names: list):
        print("%%% CALCULATING SERVER TS DIFF %%%")
            
        diffs = []
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            ds = self.calculate_server_ts_diff_for_user(user_name, path)
            
            diffs.append(ds)
            
        
        #pp.pprint(diffs)
        
        mins = []
        
        for dfs in diffs:
            mins.append(np.min(np.array(dfs)))
            
        i = 0
        for user_name in team_names:
            print("DIFF MIN FOR {}: {}".format(user_name, mins[i]))
            i += 1
            
        print("%%% DONE! %%%")    
        
        return mins
        
    def generate_DRES_logs(self, team_name: str, team_names: list):
        print("%%% GENERATING DRES LOG FILES %%%")
            
        for user_name in team_names:
            path = config.path(user_name)
            
            print("---\n\t +++ {} +++ \n\tDATA: {} \n".format(user_name, path))
                
            self.generate_DRES_results_for_user(team_name, user_name, path)
            
        print("%%% DONE! %%%")
        

    
    def calculate_server_ts_diff_for_user(self, user_name, path):
        dir = config.dir_names()["actions"]
            
        diffs = []
        
        if (self.verbose):
            print("\t--- DIR: {} ---".format(dir))

        full_path = os.path.join(path, dir)
        
        for filename in os.listdir(full_path): 
            actions = JsonLog.parse_file(full_path, filename)
            
            for a in actions:
                ser_ts = a["metadata"]["serverTimestamp"]
                loc_ts = a["metadata"]["timestamp"]
                diff = ser_ts - loc_ts
                
                if ser_ts < 10000:
                    continue
                
                diffs.append(diff)

        if (self.verbose):
            print("\t--- DONE. ---")
        
            
        return diffs
    
    def generate_DRES_results_for_user(self, team_name, user_name, path):
        dir = config.dir_names()["requests"]
        out_dir = config.out_dir("dres")
        
        
            
        if (self.verbose):
            print("\t--- DIR: {} ---".format(dir))
            print("\t--- OUT: {} ---".format(out_dir))

        full_path = os.path.join(path, dir)
        
        # For each file in the 'requests' directory
        for filename in os.listdir(full_path): 
            if (not filename.endswith("result.json")):
                continue
                
            file_json = JsonLog.parse_file(full_path, filename)
            stinfo = os.stat(os.path.join(full_path, filename))                
            
            ts = file_json["timestamp"]
            file_json["request"]["timestamp"] = ts
            
            user_out_dir = os.path.join(out_dir, team_name, user_name)
            os.makedirs(user_out_dir, exist_ok = True)
            
            fpth = os.path.join(user_out_dir, "{}.json".format(ts))
            
            
            with open(fpth, "w") as ofs:
                ofs.write(json.dumps(file_json["request"], indent = 4))
                
            os.utime(fpth ,(stinfo.st_atime, stinfo.st_mtime))
                
        if (self.verbose):
            print("\t--- DONE. ---")
        
    def validate_user(self, user_name, path):
        for _, dir in config.dir_names().items():
            
            if (self.verbose):
                print("\t--- DIR: {} ---".format(dir))
                
            full_path = os.path.join(path, dir)
            for filename in os.listdir(full_path): 
                
                if (filename.endswith("log")):
                     SummaryLog.parse_file(full_path, filename)   
                else:
                    JsonLog.parse_file(full_path, filename)
                    
            if (self.verbose):
                print("\t--- DONE. ---")
                
    def print_tasks(self):
        self.tasks.print()

In [14]:
class TaskTarget:
    def __init__(self, video_ID, fnum_from, fnum_to):
        self._video_ID = video_ID
        self._fnum_from = fnum_from
        self._fnum_to = fnum_to
        
    def interval(self):
        return (self._fnum_from, self._fnum_to)
    
    def video_ID(self):
        return self._video_ID
    
    def __str__(self):
        s = "*** TaskTarget ***\n"
        s += f"\t\tvideo_ID: {self.video_ID()}\n"
        s += f"\t\tinterval: ({self.interval()[0]}, {self.interval()[1]})\n"
        
        return s
    

class Task:
    def __init__(self, name, type, from_ts, to_ts, 
                 text=None, target:TaskTarget = None):
        self._name = name
        
        # type \in { A, V, T }
        self._type = type
        self._from_ts = from_ts
        self._to_ts = to_ts
        
        self._text = text
        self._target = target
        
        
        if (self._type == "V" and self.text() != None):
            raise Exception("Visual tasks have no texts.")
        
        if (self._type == "A" and self.target() != None):
            raise Exception("AVS task cannot have target.")
        
    def name(self):
        return self._name
    
    def type(self):
        return self._type
    
    def timestamps(self):
        return (self._from_ts, self._to_ts)
    
    def times(self):
        tss = self.timestamps()
        return (utils.from_UNIX(tss[0]), utils.from_UNIX(tss[1]))
    
    def text(self):
        return self._text
    
    def target(self):
        return self._target
    
    
    def __str__(self):
        s = "\t*** Task ***\n"
        s += f"\tname: {self.name()}\n"
        s += f"\ttype: {self.type()}\n"
        s += f"\ttimestamps: ({self.timestamps()[0]}, {self.timestamps()[1]})\n"
        s += f"\ttimes: ({self.times()[0]}, {self.times()[1]})\n"
        s += f"\ttext: {self.text()}\n"
        s += f"\ttarget: {self.target().__str__()}\n"
        
        return s
        
    
        

class TaskDefs:
    def __init__(self, thumbs_list_filepath):
        self.tasks = []
        self.thumbs = {}

        with open(thumbs_list_filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            
            for row in csv_reader:
                v_ID = int(row[0][0:5]) + 1 ## !!!
                fnum = int(row[0][42:50])
                
                if (not v_ID in self.thumbs):
                    self.thumbs[v_ID] = {}
                    
                self.thumbs[v_ID][fnum] = row[0]
                
        #pp.pprint(self.thumbs)
        
        
    def task(self, idx:int):
        return self.tasks[idx]
        
    def task(self, name:str):
        for t in self.tasks:
            if (t.name() == name):
                return t
            
    def print(self):     
        print("***############################***")
        print("***###*** VBS 2021 TASKS ***###***")
        print("***############################***")
        
        print(">>>>>>>>>>>>> STATS <<<<<<<<<<<<<<<<")
        
        print("Number of tasks: {}".format(len(self.tasks)))
        
        # Print unique video IDs
        unique_v_IDs = set()
        for t in self.tasks:
            if (t.target() != None):
                unique_v_IDs.add(t.target().video_ID())
            
        print("Unique video IDs: ")
        for u in sorted(unique_v_IDs):
            print(u, end =", ")
        print("")
        
        print(">>>>>>>>>>>>> TASKS <<<<<<<<<<<<<<<<")
        for t in self.tasks:
            print(t.__str__() + "")
            
            if (t.target() != None):
                v_ID = t.target().video_ID()
                fr, to = t.target().interval()
            
                fnms = self.filenames(v_ID, fr, to)
                fnms = list(map(lambda x: os.path.join(config.thumbs_dir(), x), fnms))
                
                utils.print_images_row(fnms)
                
            print("------------------------------------")
            
    def filename(v_ID, fnum):
        return self.thumbs[v_ID][fnum]
    
    def filenames(self, v_ID, fr, to, limit=5):
        fnums = self.thumbs[v_ID].keys()
        
        res = []
        for n in fnums:
            if (fr <= n and n <= to):
                res.append(self.thumbs[v_ID][n])
    
    
        return res
    
    @staticmethod
    def parse_tasks(tasks_fpth, tasks_ends_fpth, mapping_fpth, thumbs_list_fpth, verbose=False):
        print("%%% PARSING TASKS %%%")
            
        
        tasks_JSON = None
        with open(tasks_fpth) as ifs:
            tasks_JSON = json.load(ifs)["tasks"]
        
        guid_to_video_ID = {}
        guid_to_FPS = {}
        with open(mapping_fpth) as ifs_mapping:
            mapping_reader = csv.reader(ifs_mapping, delimiter=',')
            for row in mapping_reader:
                if (len(row) < 3):
                    break
                guid_to_video_ID[row[0]] = int(row[1])
                guid_to_FPS[row[0]] = float(row[2])
        
        
        tdefs = TaskDefs(thumbs_list_fpth)
        
        count = 0
        with open(tasks_ends_fpth) as ifs_starts:
            starts_reader = csv.reader(ifs_starts, delimiter=',')

            for row in starts_reader:
                count +=1
                t_name = row[2]
                
                if (verbose):
                    print("\t ...Task {} parsed.".format(t_name))
                
                t = utils.find_task_def(tasks_JSON, t_name)
                t_datetime = "{}T{}".format(row[0], row[1])
                
                
                t_type =t["taskType"][0:1]
                t_tsfrom = utils.UNIX_from_datetime(t_datetime)
                t_tsto = t_tsfrom + (t["duration"] * 1000) #ms
                
                text = None
                if (t_type == "T"):
                    text = ""
                    for c in t["components"]:
                        text = c["description"]
                        
                elif (t_type == "A"):
                    text = t["components"][0]["description"]
                
                t_target = None
                if (t_type != "A"):
                    guid = t["target"]["mediaItems"][0]["mediaItem"]
                    
                    fps = guid_to_FPS[guid]
                    vid_ID = guid_to_video_ID[guid]
                    
                    fr = t["target"]["mediaItems"][0]["temporalRange"]["start"]["value"] * fps
                    to = t["target"]["mediaItems"][0]["temporalRange"]["end"]["value"] * fps
                    
                    if (t["target"]["mediaItems"][0]["temporalRange"]["start"]["unit"] != "SECONDS"):
                        raise Exception("Invalid value type.")
                    
                    t_target = TaskTarget(
                        vid_ID, fr, to
                    )
                
                task = Task(
                    t_name, 
                    t_type,
                    t_tsfrom, t_tsto,
                    text,
                    t_target
                )
                tdefs.tasks.append(task)

        print(f"%%% Parsed {count} tasks. %%%")
        print("%%% DONE! %%%")
        return tdefs

    
    
    def __str__(self):
        s = "*** TaskDefs ***\n"
        
        for t in self.tasks:
            s += t.__str__() + "\n"
        
        return s

In [15]:
data = Data()
data.print_tasks()

%%% PARSING TASKS %%%
%%% Parsed 38 tasks. %%%
%%% DONE! %%%
***############################***
***###*** VBS 2021 TASKS ***###***
***############################***
>>>>>>>>>>>>> STATS <<<<<<<<<<<<<<<<
Number of tasks: 38
Unique video IDs: 
88, 1162, 1939, 2034, 2224, 2464, 2519, 3085, 3317, 3331, 3870, 3885, 4035, 4178, 4225, 4316, 4888, 4979, 5146, 5531, 6228, 6246, 6561, 6827, 6979, 7116, 7421, 
>>>>>>>>>>>>> TASKS <<<<<<<<<<<<<<<<
	*** Task ***
	name: v21-1
	type: V
	timestamps: (1624277458765, 1624277758765)
	times: (21-06-2021 14:10:58, 21-06-2021 14:15:58)
	text: None
	target: *** TaskTarget ***
		video_ID: 4178
		interval: (1875.0, 2300.0)




------------------------------------
	*** Task ***
	name: a21-5
	type: A
	timestamps: (1624277828154, 1624278128154)
	times: (21-06-2021 14:17:08, 21-06-2021 14:22:08)
	text: Find shots of a person holding or waving a flag.
	target: None

------------------------------------
	*** Task ***
	name: t21-1
	type: T
	timestamps: (1624278204021, 1624278624021)
	times: (21-06-2021 14:23:24, 21-06-2021 14:30:24)
	text: Graffity artists spraying purple outline of the letters RTH. Letters are white/green on a wall with a blueish graffity on the left. He wears a blue jacket over a green hoodie.

	target: *** TaskTarget ***
		video_ID: 3085
		interval: (7850.0, 8425.0)




------------------------------------
	*** Task ***
	name: v21-2
	type: V
	timestamps: (1624278710026, 1624279010026)
	times: (21-06-2021 14:31:50, 21-06-2021 14:36:50)
	text: None
	target: *** TaskTarget ***
		video_ID: 1162
		interval: (23050.0, 23700.0)




------------------------------------
	*** Task ***
	name: a21-9
	type: A
	timestamps: (1624279149067, 1624279449067)
	times: (21-06-2021 14:39:09, 21-06-2021 14:44:09)
	text: Find shots of at least one person drinking beer.
	target: None

------------------------------------
	*** Task ***
	name: t21-2
	type: T
	timestamps: (1624279524639, 1624279944639)
	times: (21-06-2021 14:45:24, 21-06-2021 14:52:24)
	text: A hand opening and closing a window of a mountain hut. There are snow covered mountains outside. The weather is sunny, the shadow of the hut is visible in the snow.

	target: *** TaskTarget ***
		video_ID: 2224
		interval: (2097.9021000000002, 2337.6623400000003)




------------------------------------
	*** Task ***
	name: v21-3
	type: V
	timestamps: (1624280086318, 1624280386318)
	times: (21-06-2021 14:54:46, 21-06-2021 14:59:46)
	text: None
	target: *** TaskTarget ***
		video_ID: 5146
		interval: (179.82018, 779.22078)




...

------------------------------------
	*** Task ***
	name: a21-11
	type: A
	timestamps: (1624448946650, 1624449246650)
	times: (23-06-2021 13:49:06, 23-06-2021 13:54:06)
	text: Find shots of people skiing, shot with the camera looking into the sun (back-lit shot, possibly with lens flare).
	target: None

------------------------------------
	*** Task ***
	name: v21-18
	type: V
	timestamps: (1624449356322, 1624449656322)
	times: (23-06-2021 13:55:56, 23-06-2021 14:00:56)
	text: None
	target: *** TaskTarget ***
		video_ID: 5531
		interval: (1850.0, 2175.0)




------------------------------------
	*** Task ***
	name: v21-19
	type: V
	timestamps: (1624450582353, 1624450882353)
	times: (23-06-2021 14:16:22, 23-06-2021 14:21:22)
	text: None
	target: *** TaskTarget ***
		video_ID: 2464
		interval: (5825.0, 6325.0)




------------------------------------
	*** Task ***
	name: v21-20
	type: V
	timestamps: (1624451061608, 1624451361608)
	times: (23-06-2021 14:24:21, 23-06-2021 14:29:21)
	text: None
	target: *** TaskTarget ***
		video_ID: 6827
		interval: (4531.464, 4891.103999999999)




------------------------------------
	*** Task ***
	name: a21-6
	type: A
	timestamps: (1624451557507, 1624451857507)
	times: (23-06-2021 14:32:37, 23-06-2021 14:37:37)
	text: Find underwater shots of one or more fish.
	target: None

------------------------------------
	*** Task ***
	name: v21-21
	type: V
	timestamps: (1624451948839, 1624452248839)
	times: (23-06-2021 14:39:08, 23-06-2021 14:44:08)
	text: None
	target: *** TaskTarget ***
		video_ID: 3885
		interval: (6458.541465, 6863.13687)




------------------------------------


In [16]:

data.parse_team("SOMHunter", 
                  ["sh-patrik", "sh-vit"], 
                  verbose=True, 
                  validate_fix=False,
                  generate_DRES=False
                 )


TEAM: SOMHunter

%%% CALCULATING SERVER TS DIFF %%%
---
	 +++ sh-patrik +++ 
	DATA: ./data/sh-patrik 

	--- DIR: actions ---
	--- DONE. ---
---
	 +++ sh-vit +++ 
	DATA: ./data/sh-vit 

	--- DIR: actions ---
	--- DONE. ---
DIFF MIN FOR sh-patrik: -123
DIFF MIN FOR sh-vit: 1
%%% DONE! %%%


In [17]:
print(utils.from_UNIX(1624277458765))
print(utils.from_UNIX(1624277758765))


21-06-2021 14:10:58
21-06-2021 14:15:58


In [18]:
data.parse_team("CollageHunter", 
                  ["collage-jakub", "collage-premek"], 
                  verbose=True, 
                  validate_fix=False, 
                  generate_DRES=False
                 )

TEAM: CollageHunter

%%% CALCULATING SERVER TS DIFF %%%
---
	 +++ collage-jakub +++ 
	DATA: ./data/collage-jakub 

	--- DIR: actions ---
	--- DONE. ---
---
	 +++ collage-premek +++ 
	DATA: ./data/collage-premek 

	--- DIR: actions ---
	--- DONE. ---
DIFF MIN FOR collage-jakub: 14
DIFF MIN FOR collage-premek: 5
%%% DONE! %%%


In [19]:
data.parse_team("LegacySOMHunter", 
                  ["legacy-tereza", "legacy-franta-tomas"], 
                  verbose=True, 
                  validate_fix=False, 
                  generate_DRES=False
                 )

TEAM: LegacySOMHunter

%%% CALCULATING SERVER TS DIFF %%%
---
	 +++ legacy-tereza +++ 
	DATA: ./data/legacy-tereza 

	--- DIR: actions ---
	--- DONE. ---
---
	 +++ legacy-franta-tomas +++ 
	DATA: ./data/legacy-franta-tomas 

	--- DIR: actions ---
	--- DONE. ---
DIFF MIN FOR legacy-tereza: 3
DIFF MIN FOR legacy-franta-tomas: 1
%%% DONE! %%%
